### ETL_Project_Team_1
* Goal:  Add two new datasets to the database (Stadium Capacity & Average Ticket Price)

In [1]:
# import dependencies
import pandas as pd
from sqlalchemy import create_engine

### Add Stadium Capacity table to database 'mlb_db'

In [2]:
# read in csv needed (MLB Stadium Capicity per Team) to push to database 
capacity_file = "Final_Files/MLB_Capacity.csv"
capacity_df = pd.read_csv(capacity_file, encoding="ISO-8859-1")
capacity_df.head()


,Name,Capacity,Location,Surface,Team_LongName,Opened,Distance to center field,Type,Roof type
0,Angel Stadium,45517,"Anaheim, California",Grass,Los Angeles Angels,1966,396 feet (121 m),Modern,Open
1,Busch Stadium,45494,"St. Louis, Missouri",Grass,St. Louis Cardinals,2006,400 feet (122 m),Retro-classic,Open
2,Chase Fielddouble-dagger,48686,"Phoenix, Arizona",Artificial turf,Arizona Diamondbacks,1998,407 feet (124 m),Retro-modern,Retractable
3,Citi Field,41922,"Queens, New York",Grass,New York Mets,2009,408 feet (124 m),Retro-classic,Open
4,Citizens Bank Park,42792,"Philadelphia, Pennsylvania",Grass,Philadelphia Phillies,2004,401 feet (122 m),Retro-classic,Open


In [3]:
# Establish connection with database
rds_connection_string = "postgres:smitty77@localhost:5432/mlb_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [4]:
# see what tables exist in database
engine.table_names()

['winslosses', 'attendance', 'revenue', 'payroll', 'salary', 'looktable']

In [4]:
# pull in the look table to get keys 
look = pd.read_sql_query('Select * From looktable', con=engine)
look_df = pd.DataFrame(look)
look_df.head()

,team_key,team_longname,team_cityname,team_abrv,team_shortname
0,1,Los Angeles Dodgers,Los Angeles,LA Dodgers,Dodgers
1,2,St. Louis Cardinals,St. Louis,St. Louis,Cardinals
2,3,New York Yankees,New York,NY Yankees,Yankees
3,4,Chicago Cubs,Chicago,Chicago Cubs,Cubs
4,5,Los Angeles Angels,Los Angeles,LA Angels,Angels


In [5]:
# Create a filtered dataframe from specific columns
capacity_cols = ["Team_LongName", "Name", "Capacity"]
capacity_transformed = capacity_df[capacity_cols].copy()
capacity_transformed.head()

,Team_LongName,Name,Capacity
0,Los Angeles Angels,Angel Stadium,45517
1,St. Louis Cardinals,Busch Stadium,45494
2,Arizona Diamondbacks,Chase Fielddouble-dagger,48686
3,New York Mets,Citi Field,41922
4,Philadelphia Phillies,Citizens Bank Park,42792


In [6]:
# Rename the column headers
capacity_transformed_df = capacity_transformed.rename(columns={"Team_LongName": "team_longname",
                                                          "Name": "stadium_name",
                                                          "Capacity": "stadium_capacity"})
capacity_transformed_df.head()

,team_longname,stadium_name,stadium_capacity
0,Los Angeles Angels,Angel Stadium,45517
1,St. Louis Cardinals,Busch Stadium,45494
2,Arizona Diamondbacks,Chase Fielddouble-dagger,48686
3,New York Mets,Citi Field,41922
4,Philadelphia Phillies,Citizens Bank Park,42792


In [7]:
capacity_transformed_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 3 columns):
team_longname       30 non-null object
stadium_name        30 non-null object
stadium_capacity    30 non-null int64
dtypes: int64(1), object(2)
memory usage: 848.0+ bytes


In [11]:
# merge the capacity df with the look df
merged_capacity = pd.merge(capacity_transformed_df, look_df, on="team_longname").set_index("team_key")
merged_capacity.head()

,team_longname,stadium_name,stadium_capacity,team_cityname,team_abrv,team_shortname
team_key,,,,,,
5,Los Angeles Angels,Angel Stadium,45517,Los Angeles,LA Angels,Angels
2,St. Louis Cardinals,Busch Stadium,45494,St. Louis,St. Louis,Cardinals
17,Arizona Diamondbacks,Chase Fielddouble-dagger,48686,Arizona,Arizona,Diamondbacks
13,New York Mets,Citi Field,41922,New York,NY Mets,Mets
10,Philadelphia Phillies,Citizens Bank Park,42792,Philadelphia,Philadelphia,Phillies


In [12]:
# push the new table to the database
merged_capacity.to_sql(name='capacity', con=engine, if_exists='append', index=True)

In [13]:
# check out the datatypes in the new table 
merged_capacity.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30 entries, 5 to 3
Data columns (total 6 columns):
team_longname       30 non-null object
stadium_name        30 non-null object
stadium_capacity    30 non-null int64
team_cityname       30 non-null object
team_abrv           30 non-null object
team_shortname      30 non-null object
dtypes: int64(1), object(5)
memory usage: 1.6+ KB


In [16]:
# double check that it made it -- it did!! 
engine.table_names()

['winslosses',
 'attendance',
 'revenue',
 'payroll',
 'salary',
 'capacity',
 'looktable']

### Add Average Ticket Price table to database 'mlb_db'

In [18]:
# read in csv needed (Average Ticket Price per Team) to push to database 
avg_ticket_price_file = "Final_Files/MLB_Avg_Ticket_Price.csv"
avg_ticket_price_df = pd.read_csv(avg_ticket_price_file, encoding="ISO-8859-1")
avg_ticket_price_df.head()

,Team_LongName,Avg_Ticket_Price,Team_Key
0,Arizona Diamondbacks,59,17
1,Atlanta Braves,56,12
2,Baltimore Orioles,56,28
3,Boston Red Sox,94,7
4,Chicago Cubs,128,4


In [19]:
# see what tables exist in database
engine.table_names()

['winslosses',
 'attendance',
 'revenue',
 'payroll',
 'salary',
 'capacity',
 'looktable']

In [20]:
# pull in the look table to get keys 
look = pd.read_sql_query('Select * From looktable', con=engine)
look_df = pd.DataFrame(look)
look_df.head()

,team_key,team_longname,team_cityname,team_abrv,team_shortname
0,1,Los Angeles Dodgers,Los Angeles,LA Dodgers,Dodgers
1,2,St. Louis Cardinals,St. Louis,St. Louis,Cardinals
2,3,New York Yankees,New York,NY Yankees,Yankees
3,4,Chicago Cubs,Chicago,Chicago Cubs,Cubs
4,5,Los Angeles Angels,Los Angeles,LA Angels,Angels


In [21]:
# Create a filtered dataframe from specific columns
avg_ticket_price_cols = ["Team_LongName", "Avg_Ticket_Price"]
avg_ticket_price_transformed= avg_ticket_price_df[avg_ticket_price_cols].copy()
avg_ticket_price_transformed.head()

,Team_LongName,Avg_Ticket_Price
0,Arizona Diamondbacks,59
1,Atlanta Braves,56
2,Baltimore Orioles,56
3,Boston Red Sox,94
4,Chicago Cubs,128


In [22]:
# Rename the column headers DELETE
avg_ticket_price_transformed = avg_ticket_price_transformed.rename(columns={"Team_LongName": "team_longname",
                                                          "Avg_Ticket_Price": "avg_ticket_price"})
avg_ticket_price_transformed.head()

,team_longname,avg_ticket_price
0,Arizona Diamondbacks,59
1,Atlanta Braves,56
2,Baltimore Orioles,56
3,Boston Red Sox,94
4,Chicago Cubs,128


In [23]:
# merge the capacity df with the look df
merged_avg_ticket_price = pd.merge(avg_ticket_price_transformed, look_df, on="team_longname").set_index("team_key")
merged_avg_ticket_price.head()

,team_longname,avg_ticket_price,team_cityname,team_abrv,team_shortname
team_key,,,,,
17,Arizona Diamondbacks,59,Arizona,Arizona,Diamondbacks
12,Atlanta Braves,56,Atlanta,Atlanta,Braves
28,Baltimore Orioles,56,Baltimore,Baltimore,Orioles
7,Boston Red Sox,94,Boston,Boston,Red Sox
4,Chicago Cubs,128,Chicago,Chicago Cubs,Cubs


In [24]:
# check out the datatypes in the new table 
merged_avg_ticket_price.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30 entries, 17 to 16
Data columns (total 5 columns):
team_longname       30 non-null object
avg_ticket_price    30 non-null int64
team_cityname       30 non-null object
team_abrv           30 non-null object
team_shortname      30 non-null object
dtypes: int64(1), object(4)
memory usage: 1.4+ KB


In [25]:
# push the new table to the database
merged_avg_ticket_price.to_sql(name='avgticketprice', con=engine, if_exists='append', index=True)

In [26]:
# double check that it made it -- it did!! 
engine.table_names()

['winslosses',
 'attendance',
 'revenue',
 'payroll',
 'salary',
 'capacity',
 'avgticketprice',
 'looktable']